# Factor Backtesing

* Factor Backtesing은 주식의 시장의 수익률을 분석하는 대표적인 방법 중에 하나로 가치주 효과, 소형주 효과 모멘텀 효과, 저 위험 효과 등의 수백개의 팩터들이 주가에 어떤 영향을 주는지를 과거 주가로 시뮬레이션 하는 방법
* 주가 그 자체로 다음과 같은 팩터들을 만들 수 있으므로 이를 이용하여 Python을 이용하여 Factor Backtesting을 시행하였음
* 생성된 Factor는 다음과 같음  
    - 1M Price Momentum: 21일 데이터를 가지고 측정 
    - 2M Price Momentum: 42일 데이터를 가지고 측정 
    - 3M Price Momentum: 63일 데이터를 가지고 측정
    - 1M Vol:편의상 21일로 가정
    - 1M Mean Reversion: (1-month returns - 12-month average of monthly returns)/ (12 monthly returns)의 변동성
    - 단기 Beta:21일로 측정


# Backtesing 방법
* 종목군
    -  2017년 8월 기준으로 KOSPI200에서 196 종목을 선정함(4종목이 가격이 누락으로 미입수됨)
* 포트폴리오 구성
    - 각 패터별로 수치가 존재한 종목군을 대상으로 최상위 20%는 1분위. 차상위 20%는 2분위,...  최하위20% 종목군은 5분위 포트폴리오에 분류
    - 5개 분위 포트폴리오의 누적수익률을 계산하여 해당 Factor의 주가 결정, 유용성 여부를 판단.
* 시뮬레이션 기간
    - 2007년 2월 부터 부터 2017년까지 8월까지 대상으로 작업함
* 배당 효과는 고려하지 못하였음

## 1. 주가 데이터 생성 

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt
import pandas as  pd
import numpy as np
from datetime import datetime
import seaborn as sns


In [2]:
#텍스트 파일에서 주식 데이터를 가져옴


In [3]:
path_s = "/home/bono/local_drive_u/88_alldata/stock/"
df_prices = pd.read_csv(path_s + "yahoo_prices.csv")
df_prices.rename(columns={'Date':'date','sname':'asset','Cl':'close'},inplace=True)
df_prices = df_prices[['date','asset','close','scode']]

In [4]:
# KOSPI 200 주식과 KOSPI 지수를 결합
df_kospi = pd.read_csv(path_s + "kospi.csv")
df_kospi = df_kospi[['Date','Close']].rename(columns = {'Date':'date','Close':'close'})
df_kospi['asset'] = 'KOSPI'
df_kospi['scode'] = 'KOSPI'
df_kospi['close'] = df_kospi['close'].interpolate()
df_kospi = df_kospi[(df_kospi['date'] >= df_prices['date'].min()) & (df_kospi['date'] <= df_prices['date'].max())]
df_prices = pd.concat([df_prices,df_kospi])

/home/bono/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
df_gics = pd.read_excel(path_s + "gics.xlsx", converters={'scode':str})

df_stock_list= pd.read_excel(path_s+ "krxs.xls", converters={'Code':str})
#gics_list= pd.read_excel("gics.xlsx", converters={'scode':str})


# new_list = stock_list.merge(gics_list, left_on='Code', right_on='scode')[['Code','Issue Name','업종']]
# new_list.rename(columns={'Code':'code','Issue Name':'ename','업종':'sector'},inplace=True)
# new_list['scode'] = new_list['code'] + '.KS'
# df_prices = df_prices.merge(new_list,how='left')

In [6]:
df_prices.tail()

,asset,close,date,scode
5128,KOSPI,2365.469971,2017-09-12,KOSPI
5129,KOSPI,2360.179932,2017-09-13,KOSPI
5130,KOSPI,2377.659912,2017-09-14,KOSPI
5131,KOSPI,2386.070068,2017-09-15,KOSPI
5132,KOSPI,2418.209961,2017-09-18,KOSPI


In [7]:
# #월말 일자 계산
# df_prices['month'] =  df_prices['date'].str[:7]
# port_m = df_prices.groupby('month').agg({'date' : [np.min, np.max]}).reset_index()
# port_m.columns = pd.Index([e[0] + e[1] for e in port_m.columns.tolist()])
# port_m.head()
# #df_prices_pv= df_prices.pivot_table(values= 'close', index ='date', columns= ['sector','asset'])
# #업종 정보를 가져옴
# #df_prices_pv

# 2. 팩터 데이터 생성



In [8]:
#mo_ret.describe()

In [9]:
df_prices = df_prices.drop_duplicates(['date','asset'])

In [10]:
df_prices.head()

,asset,close,date,scode
0,우리은행,21486.900391,2007-01-02,000030.KS
1,우리은행,21039.300781,2007-01-03,000030.KS
2,우리은행,20591.599609,2007-01-04,000030.KS
3,우리은행,20342.900391,2007-01-05,000030.KS
4,우리은행,19994.800781,2007-01-08,000030.KS


In [11]:
#df_prices.set_index(['date','asset'])[['close']].groupby(level =[0,1]).count().query("close >=2")
df_prices = df_prices.set_index(['date','asset'])['close'].unstack()

In [12]:
prices_piv= df_prices.unstack().reset_index()

In [13]:
history_returns = pd.DataFrame(index=pd.MultiIndex.from_product(
    [prices_piv['date'].unique(), prices_piv['asset'].unique()], names=['date', 'asset']))

In [14]:
periods = [21, 42, 63]
for period in periods:
    delta = df_prices.pct_change(period)
    history_returns[period] = delta.stack()

history_returns.rename(columns={21:'m1_ret',42:'m2_ret',63:'m3_ret'},inplace=True)

In [15]:
history_returns.dropna().tail()

m1_ret    m2_ret    m3_ret
date       asset                              
2017-09-18 현대홈쇼핑 -0.076642 -0.052434 -0.023166
           호텔신라  -0.064309 -0.023490  0.048649
           효성    -0.043750 -0.086567 -0.097345
           후성     0.143519  0.092920  0.199029
           휴켐스   -0.029880 -0.045098 -0.075901

In [16]:
#1M_vol 계산
#samsung['Volatility']= pd.rolling_std(samsung['Log_Ret'], window=252) * np.sqrt(252)
#prices = df['Cl'].unstack()
#pct_change
#pd.rolling_std(prices.pct_change(1),window=21) * np.sqrt(252)
vol_delta = df_prices.pct_change(1).rolling(center=False,window=21).std()* np.sqrt(252)


In [17]:
vol_delta.tail(10)

asset,AK홀딩스,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,GKL,GS,GS건설,...,현대위아,현대제철,현대중공업,현대차,현대해상,현대홈쇼핑,호텔신라,효성,후성,휴켐스
date,,,,,,,,,,,,,,,,,,,,,
2017-09-05,0.125309,0.305524,0.242178,0.203260,0.458267,0.250573,0.146872,0.218702,0.266751,0.220687,...,0.209956,0.268602,0.282563,0.225468,0.309619,0.260076,0.259828,0.336106,0.391609,0.164496
2017-09-06,0.124593,0.303962,0.243010,0.198635,0.460126,0.237292,0.143215,0.216305,0.267356,0.241271,...,0.227599,0.271738,0.299481,0.226117,0.308036,0.254002,0.253832,0.325859,0.374621,0.162742
2017-09-07,0.167032,0.304009,0.257623,0.199712,0.461689,0.186158,0.146363,0.211390,0.262668,0.240687,...,0.232962,0.255122,0.306313,0.218470,0.306789,0.247249,0.261201,0.317808,0.372888,0.164794
2017-09-08,0.168092,0.303588,0.256355,0.197454,0.460329,0.185587,0.146012,0.211483,0.268531,0.236145,...,0.311006,0.252861,0.308365,0.217392,0.307722,0.239367,0.269955,0.318118,0.344580,0.159503
2017-09-11,0.168037,0.307304,0.257227,0.234897,0.460062,0.176808,0.148243,0.220609,0.260941,0.253616,...,0.308477,0.252764,0.294046,0.217046,0.298834,0.257307,0.270804,0.327704,0.467574,0.165827
2017-09-12,0.173052,0.277955,0.269375,0.205505,0.316492,0.142740,0.143461,0.206312,0.260878,0.255904,...,0.310973,0.232517,0.297289,0.210417,0.310161,0.240889,0.306570,0.327081,0.465182,0.157839
2017-09-13,0.181333,0.284511,0.256630,0.213583,0.307270,0.144922,0.153768,0.209276,0.245992,0.265816,...,0.298193,0.237818,0.298326,0.202149,0.321475,0.190924,0.310661,0.330654,0.452534,0.158635
2017-09-14,0.182029,0.268313,0.267390,0.205961,0.318800,0.145001,0.153674,0.209366,0.196715,0.265483,...,0.286428,0.221120,0.284363,0.221039,0.315391,0.188048,0.311080,0.283177,0.454804,0.174307
2017-09-15,0.188328,0.271876,0.263067,0.232546,0.326793,0.137089,0.163477,0.208424,0.203330,0.265483,...,0.344761,0.188920,0.317639,0.221051,0.321404,0.187897,0.307638,0.283585,0.459134,0.156383


In [18]:
history_returns['vol21'] = vol_delta.stack()
history_returns.tail()

m1_ret    m2_ret    m3_ret     vol21
date       asset                                        
2017-09-18 현대홈쇼핑 -0.076642 -0.052434 -0.023166  0.211246
           호텔신라  -0.064309 -0.023490  0.048649  0.313163
           효성    -0.043750 -0.086567 -0.097345  0.277597
           후성     0.143519  0.092920  0.199029  0.452040
           휴켐스   -0.029880 -0.045098 -0.075901  0.170336

In [19]:
#mean_reversion

mr_delta = ((df_prices.pct_change(21)- df_prices.pct_change(252)/12)) /  (df_prices.pct_change(1).rolling(center=False,window=252).std()* np.sqrt(252))
# mr_delta = mr_delta/vol_delta
history_returns['mr1'] = mr_delta.stack()

In [20]:
# beta 21
def roll(df, w):
    
    roll_array = np.dstack([df.values[i:i+w, :] for i in range(len(df.index) - w + 1)]).T

    panel = pd.Panel(roll_array, 
                     items=df.index[w-1:],
                     major_axis=df.columns,
                     minor_axis=pd.Index(range(w), name='roll'))

    return panel.to_frame().unstack().T.groupby(level=0)

In [21]:
def beta(df):
    # first column is the market
    X = df.values[:, [0]]
    # prepend a column of ones for the intercept
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # matrix algebra
    b = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(df.values[:, 1:])
    return pd.Series(b[1], df.columns[1:], name='Beta')

In [22]:
df_ret__ = df_prices.pct_change(1)
df_ret__ = df_ret__.fillna(0)

rdf = roll(df_ret__, 21)
betas = rdf.apply(beta)

history_returns['beta21'] = betas.stack()

/home/bono/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  after removing the cwd from sys.path.


In [23]:
history_returns.unstack()['beta21'].tail()

asset,AK홀딩스,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,GKL,GS,GS건설,...,현대위아,현대제철,현대중공업,현대차,현대해상,현대홈쇼핑,호텔신라,효성,후성,휴켐스
date,,,,,,,,,,,,,,,,,,,,,
2017-09-12,NaN,0.304249,0.248925,0.004595,0.558825,0.006498,0.451542,0.559238,0.093885,0.013679,...,-0.145480,-0.310111,0.690109,0.053156,0.409288,0.227599,0.905658,-0.239579,0.006273,0.022270
2017-09-13,NaN,0.166430,0.221335,-0.091678,0.359248,-0.040603,0.319914,0.446410,0.063036,0.130968,...,-0.114243,-0.365533,0.686772,-0.017773,0.233484,0.068197,0.741024,-0.300956,-0.037089,-0.007170
2017-09-14,NaN,0.164708,0.263550,-0.090987,0.398596,-0.046668,0.316354,0.434783,0.022634,0.137780,...,-0.141955,-0.378330,0.686623,0.027803,0.251768,0.074944,0.744195,-0.314972,-0.002174,0.028990
2017-09-15,NaN,0.215324,0.210674,-0.232849,0.268500,-0.012821,0.215291,0.377163,-0.056442,0.159100,...,-0.396101,-0.330109,0.448314,0.031057,0.124448,0.078466,0.706737,-0.349354,0.132346,0.058043
2017-09-18,NaN,0.250222,0.259854,-0.260335,0.438372,-0.025640,0.207505,0.358630,-0.089130,0.151576,...,-0.390276,-0.318176,0.387357,0.048724,0.143475,0.013627,0.739855,-0.339153,0.181028,0.108578


In [24]:
## 팩터 데이터에 의한 랭크

quantiles = 5 
#np.arange(0,201,40)
merged_data = history_returns.copy()
merged_data = merged_data.dropna()

def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

def quantile_calc(x, _quantiles):
        return pct_rank_qcut(x, _quantiles) 

In [25]:
grouper = [merged_data.index.get_level_values('date')]
for cm in merged_data.columns:
    factor_quantile = merged_data.groupby(grouper)[cm].apply(quantile_calc, quantiles)
    merged_data['rank_' + cm] = factor_quantile
    print('rank_' + cm)


/home/bono/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  # Remove the CWD from sys.path while we load stuff.


rank_m1_ret
rank_m2_ret
rank_m3_ret
rank_vol21
rank_mr1
rank_beta21


In [26]:
merged_data.tail()

m1_ret    m2_ret    m3_ret     vol21       mr1    beta21  \
date       asset                                                               
2017-09-18 현대홈쇼핑 -0.076642 -0.052434 -0.023166  0.211246 -0.297912  0.013627   
           호텔신라  -0.064309 -0.023490  0.048649  0.313163 -0.153661  0.739855   
           효성    -0.043750 -0.086567 -0.097345  0.277597 -0.195144 -0.339153   
           후성     0.143519  0.092920  0.199029  0.452040  0.145830  0.181028   
           휴켐스   -0.029880 -0.045098 -0.075901  0.170336 -0.171134  0.108578   

                  rank_m1_ret  rank_m2_ret  rank_m3_ret  rank_vol21  rank_mr1  \
date       asset                                                                
2017-09-18 현대홈쇼핑            2            3            3           2         1   
           호텔신라             2            4            4           4         3   
           효성               3            2            2           3         2   
           후성               5            5            5           5         5   
           휴켐스              3            3            3           1         3   

                  rank_beta21  
date       asset               
2017-09-18 현대홈쇼핑            2  
           호텔신라             5  
           효성               1  
           후성               4  
           휴켐스              3

In [27]:
factor_data = merged_data.copy()
factor_data.describe()

,m1_ret,m2_ret,m3_ret,vol21,mr1,beta21,rank_m1_ret,rank_m2_ret,rank_m3_ret,rank_vol21,rank_mr1,rank_beta21
count,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000,423853.000000
mean,0.009592,0.019357,0.028559,0.376553,-0.002011,0.145971,3.009253,3.009180,3.009149,3.009164,3.009164,3.009164
std,0.120456,0.175205,0.219816,0.216747,0.267132,0.297524,1.413980,1.413994,1.413980,1.413989,1.413989,1.413989
min,-0.668758,-0.775974,-0.809888,0.000000,-1.264731,-3.755628,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,-0.058288,-0.079861,-0.096250,0.258879,-0.172726,-0.029567,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,0.001076,0.003871,0.005721,0.335674,-0.017473,0.110298,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,0.068376,0.100993,0.126582,0.437977,0.153746,0.292695,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,3.533125,5.717850,6.261411,12.297447,3.363545,5.542502,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [28]:
factor_df = factor_data.unstack()

In [29]:
#factor_df.to_pickle(path_s +'factor_df.p')
#factor_df = pd.read_pickle(path_s +'factor_df.p')

## 3 포트폴리오 성과 측정



In [30]:
factor_data = factor_df.stack()

In [31]:
df_prices.shape

(2667, 201)

In [32]:
stock_data = df_prices.unstack().copy()

In [33]:
stock_df = stock_data.unstack(-2)

In [34]:
factor_data.head()

beta21    m1_ret    m2_ret    m3_ret       mr1  \
date       asset                                                      
2008-01-10 CJ      0.449269 -0.040151 -0.141414  0.020000 -0.117888   
           CJ CGV -0.094575  0.081784 -0.016892 -0.177966  0.270208   
           CJ대한통운 -0.725867  0.054190  0.705403 -0.112033  0.011379   
           GS      0.286509 -0.047619 -0.181287  0.102362 -0.277543   
           GS건설    0.189980  0.031974  0.014287  0.023057 -0.114307   

                   rank_beta21  rank_m1_ret  rank_m2_ret  rank_m3_ret  \
date       asset                                                        
2008-01-10 CJ              5.0          3.0          2.0          4.0   
           CJ CGV          1.0          5.0          4.0          2.0   
           CJ대한통운          1.0          5.0          5.0          3.0   
           GS              4.0          3.0          2.0          5.0   
           GS건설            3.0          5.0          5.0          5.0   

                   rank_mr1  rank_vol21     vol21  
date       asset                                   
2008-01-10 CJ           4.0         2.0  0.311053  
           CJ CGV       5.0         2.0  0.346385  
           CJ대한통운       5.0         5.0  5.406550  
           GS           2.0         4.0  0.458774  
           GS건설         4.0         5.0  0.632052

In [35]:
factor_df = factor_data.unstack()

In [52]:
factor_df.tail()

beta21                                                    \
asset             CJ    CJ CGV    CJ대한통운        GS      GS건설    JW중외제약   
date                                                                     
2017-09-12  0.004595  0.558825  0.006498  0.093885  0.013679  0.386789   
2017-09-13 -0.091678  0.359248 -0.040603  0.063036  0.130968  0.278963   
2017-09-14 -0.090987  0.398596 -0.046668  0.022634  0.137780  0.265449   
2017-09-15 -0.232849  0.268500 -0.012821 -0.056442  0.159100  0.178438   
2017-09-18 -0.260335  0.438372 -0.025640 -0.089130  0.151576  0.165731   

                                                      ...        vol21  \
asset            KCC     KOSPI        KT      KT&G    ...         현대로템   
date                                                  ...                
2017-09-12  0.138453  0.159317  0.197549  0.022069    ...     0.205633   
2017-09-13  0.066468  0.128726  0.205659 -0.120571    ...     0.209085   
2017-09-14  0.087703  0.141769  0.159064 -0.140447    ...     0.218854   
2017-09-15 -0.016231  0.121937  0.083550 -0.368141    ...     0.221859   
2017-09-18  0.016592  0.146571  0.071045 -0.400113    ...     0.226600   

                                                                        \
asset            종근당      코스맥스    BGF리테일      쿠쿠전자   삼성에스디에스        만도   
date                                                                     
2017-09-12  0.274553  0.393306  0.277955  0.263936  0.271107  0.267661   
2017-09-13  0.270709  0.401398  0.284511  0.265204  0.273122  0.255214   
2017-09-14  0.268413  0.439789  0.268313  0.255316  0.273848  0.251630   
2017-09-15  0.256677  0.437513  0.271876  0.241862  0.274677  0.262249   
2017-09-18  0.255944  0.443272  0.264139  0.241881  0.277022  0.268116   

                                          
asset           삼성물산       이노션    LIG넥스원  
date                                      
2017-09-12  0.201629  0.226123  0.329778  
2017-09-13  0.206844  0.226222  0.325891  
2017-09-14  0.198721  0.244514  0.327230  
2017-09-15  0.198721  0.244555  0.328939  
2017-09-18  0.320583  0.242429  0.339302  

[5 rows x 2316 columns]

In [37]:

#월말 일#자 계산
df_prices_2 = df_prices.copy()
df_prices_2['month'] =  df_prices_2.index.str[:7]
port_month_info = df_prices_2.reset_index().groupby('month')['date'].agg(['min','max'])

In [38]:
port_month_info.head()

,min,max
month,,
2007-01,2007-01-02,2007-01-31
2007-02,2007-02-01,2007-02-28
2007-03,2007-03-01,2007-03-30
2007-04,2007-04-02,2007-04-30
2007-05,2007-05-02,2007-05-31


In [39]:
stock_df.index = pd.to_datetime(stock_df.index)
factor_df.index = pd.to_datetime(factor_df.index)

In [40]:
#stock_df.resample('bm')
stock_montly_df = stock_df.resample('BM').last()
factor_monthly_df = factor_df.resample('BM').last()

In [41]:
stock_montly_df.tail()

asset,AK홀딩스,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,GKL,GS,GS건설,...,현대위아,현대제철,현대중공업,현대차,현대해상,현대홈쇼핑,호텔신라,효성,후성,휴켐스
date,,,,,,,,,,,,,,,,,,,,,
2017-05-31,68800.0,139500.0,10000.0,211500.0,84500.0,192500.0,381500.0,22200.0,72900.0,30700.0,...,69000.0,59200.0,173000.0,163000.0,37750.0,137000.0,60800.0,168000.0,8530.0,25450.0
2017-06-30,74200.0,101000.0,10950.0,189500.0,73600.0,180500.0,361500.0,22400.0,68200.0,30450.0,...,69800.0,62200.0,176500.0,159500.0,39350.0,131000.0,57100.0,167500.0,10550.0,26750.0
2017-07-31,71700.0,92400.0,11300.0,194000.0,69300.0,186500.0,370000.0,22950.0,75400.0,31850.0,...,69000.0,62700.0,174500.0,145000.0,45400.0,138000.0,65000.0,167500.0,11000.0,24950.0
2017-08-31,70200.0,91200.0,9860.0,177000.0,65400.0,169000.0,362500.0,23100.0,70600.0,28500.0,...,70000.0,57700.0,148500.0,140500.0,46200.0,137500.0,61900.0,158500.0,11200.0,23950.0
2017-09-29,69400.0,82800.0,10200.0,176500.0,68200.0,161500.0,350500.0,22050.0,66000.0,27400.0,...,61700.0,52600.0,138000.0,137000.0,45100.0,126500.0,58200.0,153000.0,12350.0,24350.0


In [42]:
stock_montly_df.tail()

asset,AK홀딩스,BGF리테일,BNK금융지주,CJ,CJ CGV,CJ대한통운,CJ제일제당,GKL,GS,GS건설,...,현대위아,현대제철,현대중공업,현대차,현대해상,현대홈쇼핑,호텔신라,효성,후성,휴켐스
date,,,,,,,,,,,,,,,,,,,,,
2017-05-31,68800.0,139500.0,10000.0,211500.0,84500.0,192500.0,381500.0,22200.0,72900.0,30700.0,...,69000.0,59200.0,173000.0,163000.0,37750.0,137000.0,60800.0,168000.0,8530.0,25450.0
2017-06-30,74200.0,101000.0,10950.0,189500.0,73600.0,180500.0,361500.0,22400.0,68200.0,30450.0,...,69800.0,62200.0,176500.0,159500.0,39350.0,131000.0,57100.0,167500.0,10550.0,26750.0
2017-07-31,71700.0,92400.0,11300.0,194000.0,69300.0,186500.0,370000.0,22950.0,75400.0,31850.0,...,69000.0,62700.0,174500.0,145000.0,45400.0,138000.0,65000.0,167500.0,11000.0,24950.0
2017-08-31,70200.0,91200.0,9860.0,177000.0,65400.0,169000.0,362500.0,23100.0,70600.0,28500.0,...,70000.0,57700.0,148500.0,140500.0,46200.0,137500.0,61900.0,158500.0,11200.0,23950.0
2017-09-29,69400.0,82800.0,10200.0,176500.0,68200.0,161500.0,350500.0,22050.0,66000.0,27400.0,...,61700.0,52600.0,138000.0,137000.0,45100.0,126500.0,58200.0,153000.0,12350.0,24350.0


In [44]:
stock_monthly_ret_data = stock_montly_df.pct_change().stack()
stock_monthly_ret_data.tail()

date        asset
2017-09-29  현대홈쇼핑   -0.080000
            호텔신라    -0.059774
            효성      -0.034700
            후성       0.102679
            휴켐스      0.016701
dtype: float64

In [45]:
# 측정기준일과. 성과측정일을 계산함.
# 포트폴리오 리배런싱일은 endday , 성과 측정은 perf_check_day 

port_month_info.rename(columns = {'min':'startday', 'max':'endday'}, inplace = True)
port_month_info['perf_check_day'] = port_month_info['endday'].shift(-1)

In [46]:
factor_data['rank_m1_ret'].head()

date        asset 
2008-01-10  CJ        3.0
            CJ CGV    5.0
            CJ대한통운    5.0
            GS        3.0
            GS건설      5.0
Name: rank_m1_ret, dtype: float64

In [50]:
port_month_info.head()

,startday,endday,perf_check_day
month,,,
2007-01,2007-01-02,2007-01-31,2007-02-28
2007-02,2007-02-01,2007-02-28,2007-03-30
2007-03,2007-03-01,2007-03-30,2007-04-30
2007-04,2007-04-02,2007-04-30,2007-05-31
2007-05,2007-05-02,2007-05-31,2007-06-29


In [51]:
#port_before 에는 리밸런스 날 의사결정을 할 수 있는 개별 팩터에 대한 주식의 순위 정보가 있음 
#perf_after 에는 다음 리밸런스 날 까지 각 주식의 월간 수익률 정보가 있음

appended_data = []

facotr_list = ['rank_beta21', 'rank_m1_ret', 'rank_m2_ret', 'rank_m3_ret', 'rank_mr1', 'rank_vol21']
#factor_rank ='rank_m1_ret'
for factor_rank in facotr_list:
    for index, row in port_month_info.iterrows():
        #print(row['perf_check_day'])
        port_before = factor_data[factor_data.index.get_level_values(0) == row['endday']][factor_rank]
        #print(row)
        #print(port_before.head(10))
        perf_after = stock_monthly_ret_data[stock_monthly_ret_data.index.get_level_values(0)  == row['perf_check_day']]

        merge_factor_ret = port_before.reset_index().merge(perf_after.reset_index(), left_on= 'asset', right_on='asset')
        
        appended_data.append(merge_factor_ret)
    appended_df = pd.concat(appended_data, axis=0)
    print(appended_df.groupby(factor_rank).agg({0:['count','mean']}))

    plt.show()

/home/bono/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



                0          
            count      mean
rank_beta21                
1.0          3670  0.007426
2.0          3720  0.009640
3.0          3709  0.008542
4.0          3720  0.011018
5.0          3755  0.012763
                0          
            count      mean
rank_m1_ret                
1.0          3670  0.009329
2.0          3717  0.009714
3.0          3712  0.008969
4.0          3719  0.007880
5.0          3756  0.013517
                0          
            count      mean
rank_m2_ret                
1.0          3670  0.011850
2.0          3719  0.007755
3.0          3710  0.008476
4.0          3719  0.009553
5.0          3756  0.011823
                0          
            count      mean
rank_m3_ret                
1.0          3670  0.009411
2.0          3719  0.008138
3.0          3711  0.008386
4.0          3719  0.010485
5.0          3755  0.012995
             0          
         count      mean
rank_mr1                
1.0       3670  0.009217
2.0 

## 4. Summary

 1. 가격 모멘텀이 조금 나타남
 2. 변동성은 크게 영향을 줌
 3. 단기 베타는 영향을 주지 않음
 4. 수익률이 크게 변한 주식에 대해 추가적인 분석이 필요함 
